In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imgrvt as rvt
import pims
import trackpy as tp
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, WhiteKernel
from matplotlib.patches import Circle
%config InlineBackend.figure_format='retina'
from scipy.signal import find_peaks
from scipy.signal import peak_widths
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rc('xtick', labelsize=8)      # fontsize of the tick labels
plt.rc('ytick', labelsize=8)      # fontsize of the tick labels 
from sympy import *
cmap = matplotlib.colormaps['viridis']
cmap
from colormath.color_objects import XYZColor, sRGBColor
from colormath.color_conversions import convert_color
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import time
from sklearn.gaussian_process.kernels import RBF
from scipy.interpolate import CubicSpline
tab10 = plt.get_cmap("tab10")
viridis = plt.get_cmap("viridis")
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import LSQUnivariateSpline
import seaborn as sns

In [ ]:
## read the green channel of the RGB image
@pims.pipeline
def gray(image):
    return image[:, :, 1]

# read all the images corresponding to different time points at a specific location within the capillary
frames = gray(pims.open('/*.png'))
Lt = len(frames)
frames[0]


In [ ]:
Lt

In [ ]:
def plot_comparison(src, results, titles=None): # plot several images to compare different RVT (radial variance transform) parameters
    nres=len(results)
    _,axes=plt.subplots(1,nres+1,figsize=(4*nres+4,3))
    axes[0].imshow(src,cmap="gray")
    axes[0].set_title("Source")
    for ax,res,t in zip(axes[1:],results,titles or [None]*nres):
        ax.imshow(res,cmap="inferno")
        if t:
            ax.set_title(t)

In [ ]:
Ly,Lx = frames[0].shape
start = 45
xsize = 2080
ysize = 1552
xmin = 0
ymin = 0
img = frames[start][ymin:ymin+ysize, xmin:xmin+xsize]
img

In [ ]:
rmin = 30
rmax = 150
highpass_size = 1
kind = 'normalized'
img_rvt = rvt.rvt(img, rmin=rmin, rmax=rmax, highpass_size=highpass_size, kind=kind)
plot_comparison(img, [img_rvt], ["RVT"])

In [ ]:
plt.imshow(img_rvt)
plt.colorbar();

In [ ]:
diameter = 3
separation = 50
f = tp.locate(img_rvt, diameter, preprocess=False, separation=separation)
tp.annotate(f, img, plot_style={'marker':'.','markersize':4});

In [ ]:
n_bins = 16;
range = (0,1.6)
plt.hist(f['mass'], bins = n_bins, range = range, rwidth = 0.9)
plt.xlabel('mass');
plt.ylabel('count');


plt.ylim(0,100)
plt.xticks(fontsize=14);
plt.yticks(fontsize=14);
plt.hist(img_rvt, n_bins, range = range)

In [ ]:
diameter = 3
minmass = 0.1
separation = 50
f = tp.locate(img_rvt, diameter, preprocess=False, minmass=minmass, separation=separation) #topn = 20
tp.annotate(f, img, plot_style={'marker':'.','markersize':4});

In [ ]:
f.head()

# Process Video (all Drops)

In [ ]:
start = 0
stop = 100

In [ ]:
# define parameters for the RVT function
rmin = 30
rmax = 90
highpass_size = 1
kind = 'normalized'
diameter = 3
minmass = 0.1
separation = 50
wsize = 120

In [ ]:
# only adds outputs of rvt.rvt fucntion that identifies drop centers to rvt.imgs folder 
imgs = []
rvt_imgs = []
finish = len(frames)
for frame in frames[start:stop]:
    img = frame[ymin:ymin+ysize, xmin:xmin+xsize]
    rvt_img = rvt.rvt(img, rmin=rmin, rmax=rmax, highpass_size=highpass_size, kind=kind)
    imgs.append(img)
    rvt_imgs.append(rvt_img)
    if len(rvt_imgs)%50==0:
        print(len(rvt_imgs))
        
# % is Modulo Operator - divides LHS by RHS

In [ ]:
len(rvt_imgs)

In [ ]:
# tracks drops centers for all images present in rvt.imgs
df_tp_batch = tp.batch(rvt_imgs, diameter, preprocess=False, minmass=minmass, separation=separation)
df_tp_batch.head()

In [ ]:
# link particles using tp.link_df
linked_droplets = tp.link_df(df_tp_batch, search_range=70, memory=3)

# search_rangefloat or tuple the maximum distance features can move between frames, optionally per dimension
# memoryinteger, optional - the maximum number of frames during which a feature can vanish, then reappear nearby, and be considered the same particle. 0 by default.

# remove drops near the edge
idx = (linked_droplets['x'] >= wsize ) & (linked_droplets['x'] < xsize - wsize - 2) & (linked_droplets['y'] >= wsize) & (linked_droplets['y'] < ysize - wsize - 2)
linked_droplets = linked_droplets[idx]

In [ ]:
ax = tp.plot_traj(linked_droplets, label=True, superimpose=imgs[0])

# Radius calculation

In [ ]:
def calc_drop_size3(img, row, wsize, theta = 1, sigma = 5):
    # identify drop
    yc = int(np.round(row['y'])) # drop center pixel accuracy
    xc = int(np.round(row['x'])) 
    dy = row['y'] - yc
    dx = row['x'] - xc
    
    if (yc - wsize < 0) or (yc + wsize + 1 >= ysize) or (xc - wsize < 0) or (xc + wsize + 1 >= xsize):
        return np.nan
    
    # make a window
    yw = np.arange(-wsize, wsize + 1) - dy
    xw = np.arange(-wsize, wsize + 1) - dx
    rw = np.ndarray.flatten(np.sqrt(yw[:, None]**2 + xw[None, :]**2))
    iw = np.ndarray.flatten(img[yc - wsize:yc + wsize + 1, xc - wsize:xc + wsize + 1])
    
    num_of_knots = 30
    knot_positions = np.linspace(3, wsize, num_of_knots)

    # Performe the spline fitting (degree=k)
    isort = np.argsort(rw)
    spline = LSQUnivariateSpline(rw[isort], iw[isort], knot_positions, k=5)

    # Calculate second derivative at multiple points
    r_eval = np.linspace(3, wsize, 1000)  # Evaluate over the entire range
    spline_second_derivative = spline.derivative(2)
    second_derivative_values = spline_second_derivative(r_eval)

    # Find sign changes in the second derivative
    sign_changes = np.where(np.diff(np.sign(second_derivative_values)))[0]

    # Calculate the derivative of the spline at the inflection points
    inflection_points = r_eval[sign_changes]
    spline_derivative_values = spline.derivative()(inflection_points)

    # Find the inflection point with the maximum positive slope
    max_positive_slope_index = np.argmax(spline_derivative_values)
    rmax2 = inflection_points[max_positive_slope_index]
    imax2 = spline(rmax2)
    
    radius = rmax2
        
    return radius

In [ ]:
# Convert 'frame' column to integers in final_droplets DataFrame
linked_droplets['frame'] = linked_droplets['frame'].astype(int)

# Iterate through each unique particle
for particle in linked_droplets['particle'].unique():
    idx = (linked_droplets['particle'] == particle)
    print('particle=', particle)
    
    # Iterate through the rows for the current particle
    for index, row in linked_droplets[idx].iterrows():
        # Ensure the 'frame' value is an integer
        frame_index = int(row['frame'])
        print('frame = ', frame_index)
        wsize = 120
            
        #checkpoint
            
        if frame_index > 1:
            frame_2 = frame_index - 1
            index_2 = (linked_droplets['particle']==particle) & (linked_droplets['frame']==frame_2)
            #radius of particle in the previous frame
            radius_2 = linked_droplets[index_2]['radius'].values
            checkpoint = (wsize - 1)/pixel_per_micron
            if radius_2 > checkpoint:
                wsize = 150  
                print('changed wsize', wsize)
            
        radius = calc_drop_size3(imgs[frame_index], row, wsize)

        # Check for NaN values in the radius calculation
        if not np.isnan(radius):
            # Convert radius to microns
            pixel_per_micron = 6.76
            radius_microns = radius / pixel_per_micron

            # Update 'radius' column for the current row
            linked_droplets.at[index, 'radius'] = radius_microns

            # Print progress
            print(f"Particle {particle}, Radius: {radius_microns}")

# Check the resulting DataFrame with updated radius values
print(linked_droplets.head())



# only consider the drops that appear in frame 0

In [ ]:
particles_in_frame_0 = linked_droplets[linked_droplets['frame'] == 0]['particle'].unique()
final_droplets = linked_droplets[linked_droplets['particle'].isin(particles_in_frame_0)]

In [ ]:
# group the data by particle ID and count the number of frames each particle appears in after filtering
frame0_counts = final_droplets.groupby('particle')['frame'].nunique()
frame0counts = pd.DataFrame(frame0_counts)
frame0counts.head()


# signal and vacoule detection

In [ ]:
# upload csv file/dataframe generated above that contains the pixel location and radius of each drop present in frame 0
final_droplets = pd.read_csv('.csv')

In [ ]:
start = 0
stop = 99
frame_nos = np.arange(start, stop + 1)

all_vsignals = []

particles = final_droplets['particle'].unique()

for particle in particles:
    vsignals = []  # initialize the signals list for the current particle
    print("Processing particle:", particle)
    
    for frame in frame_nos:
        #frame_v2 = frame + 500
        idx = (final_droplets['frame'] == frame) & (final_droplets['particle'] == particle)
        img = frames[frame][ymin:ymin + ysize, xmin:xmin + xsize]
        radius = final_droplets[idx]['radius'].values * 6.76
        if not len(radius) == 0:                                                                      
            print("Processing frame:", frame)

            wsize = radius * 1.2
            wsize = int(wsize)

            yc = final_droplets['y'].values[idx]
            xc = final_droplets['x'].values[idx]

            ycPix = int(yc)
            xcPix = int(xc)

            xPix = np.arange(-wsize, wsize)
            yPix = np.arange(-wsize, wsize + 1)
            rPix = np.sqrt((xPix[:, None] + xc - xcPix) ** 2 + (yPix[None, :] + yc - ycPix) ** 2)
            thetaPix = np.arctan2(yPix[None, :] + yc - ycPix, xPix[:, None] + xc - xcPix)

            rMax = int(wsize / np.sqrt(2)) # rMax is 85% of the radius
            rBins = np.arange(2, rMax + 1)
            means = np.zeros((rMax - 1,))
            variances = np.zeros((rMax - 1,))
            counts = np.zeros((rMax - 1,))
            params = np.zeros((rMax - 1, 7))
            cropped = img[ycPix - wsize:ycPix + wsize, xcPix - wsize:xcPix + wsize + 1]
                
            for i, r in enumerate(rBins):
                index = (rPix > r) & (rPix < r + 1) # concentric circle
                intensities = np.array(cropped[index])

                # calculate variances and means
                counts[i] = intensities.shape[0]
                means[i] = np.mean(intensities)
                variances[i] = np.var(intensities)

                # fit harmonic functions
                X = np.column_stack((thetaPix[index] ** 0, np.cos(thetaPix[index]), np.sin(thetaPix[index]),
                                    np.cos(2 * thetaPix[index]), np.sin(2 * thetaPix[index]),
                                    np.cos(3 * thetaPix[index]), np.sin(3 * thetaPix[index])))
                params[i, :] = np.linalg.lstsq(X, intensities, rcond=None)[0]

            vsignal = np.sum(counts * (params[:, 5] ** 2 + params[:, 6] ** 2) / np.sqrt(np.pi)) / np.sum(counts)
            #all_vsignals.append(vsignal)
            final_droplets.loc[idx, 'vsignal'] = vsignal
            print('visgnal=', radius)
        #else:
            #vsignals.append(np.nan)
    #final_droplets.loc[idx, 'vsignal'] = np.nan
            
    print(f"Particle {particle}, Frame {frame}: Signal = {vsignal}")


## plot vsignal of a single drop

In [ ]:
plt.figure(figsize=(12, 6))

particle = 14
mask = final_droplets['particle'] == particle
data = final_droplets[mask]
plt.plot(data['frame'], data['vsignal'], '-o', label=f'Particle {particle}')
plt.xlim(0,135)

plt.axhline(y=1.2)
plt.xlabel('frame', fontsize = 14)
plt.ylabel('vsignal', fontsize = 14)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

#plt.show()


In [ ]:
final_droplets['vacuole'] = np.where(final_droplets['vsignal'] > 1.2, 'yes', 'no')


In [ ]:
final_droplets.to_csv('.csv')

## Detect the beginning of vacuole formation in drops

In [ ]:
# upload csv file/dataframe generated above that contains the pixel location, radius and angular variance of each drop 
df = pd.read_csv()

In [ ]:
# filter particles with 'vacuole' = 'yes' for more than 3 frames
particle_vacuole_counts = df[df['vacuole'] == 'yes'].groupby('particle').size()
qualified_particles = particle_vacuole_counts[particle_vacuole_counts >= 3].index

# filter the original df based on selected particles
filtered_df = df[df['particle'].isin(qualified_particles)]

# find the first occurrence of vsignal > 1.2 for each qualified particle
particle_no = []
first_vsignal = []
pH_values = []
dpHdt_values = []
radius_values = []

for particle in qualified_particles:
    particle_data = filtered_df[(filtered_df['particle'] == particle) & (filtered_df['vsignal'] > 1.2)]
    if not particle_data.empty:
        particle_no.append(particle_data['particle'].iloc[0])
        
        first_vsignal.append(particle_data['vsignal'].iloc[0])
        
        pH_values.append(particle_data['pH'].iloc[0])
        
        dpHdt_values.append(particle_data['dpH/dt'].iloc[0])
        
        radius_values.append(particle_data['radius'].iloc[0])

plt.figure(figsize=(8, 6))

plt.scatter(pH_values, radius_values, label='Particles')


plt.xlabel('pH')
plt.ylabel('radius')
plt.legend()
plt.show()


In [ ]:
# Pad the list with NaN to match the length of the DataFrame
particle_no_v2 = particle_no + [np.nan] * (len(df) - len(first_vsignal))

# Add a new column named 'particle_vfirst' with the values from the padded list
df['particle_vfirst'] = particle_no_v2

pH_values_v2 = pH_values + [np.nan] * (len(df) - len(first_vsignal))
df['pH_vfirst'] = pH_values_v2

dpHdt_values_v2 = dpHdt_values + [np.nan] * (len(df) - len(first_vsignal))
df['dpHdt_vfirst'] = dpHdt_values_v2

first_vsignal_v2 = first_vsignal + [np.nan] * (len(df) - len(first_vsignal))
df['vfirst'] = first_vsignal_v2

radius_values_v2 = radius_values + [np.nan] * (len(df) - len(first_vsignal))
df['radius_vfirst'] = radius_values_v2


# Save the DataFrame back to the CSV file
df.to_csv(csv_file_path, index=False)